In [1]:
import pandas as pd
import numpy as np

# 📁 Load dataset
df = pd.read_csv("purchase_data_exe.csv")  # or "ecommerce_data_cleaned.csv" if you cleaned it earlier

print("🔹 Initial shape:", df.shape)

# --------------------------------------------
# 1️⃣ Handle Missing Values
# --------------------------------------------
print("\n🔍 Checking missing values:")
print(df.isnull().sum())

# Drop rows with missing Order ID or Customer/User ID
if 'order_id' in df.columns:
    df = df[df['order_id'].notnull()]
if 'user_id' in df.columns:
    df = df[df['user_id'].notnull()]

# Fill missing price or quantity with median (if any)
for col in ['price', 'quantity']:
    if col in df.columns and df[col].isnull().sum() > 0:
        df[col] = df[col].fillna(df[col].median())

# --------------------------------------------
# 2️⃣ Remove Duplicate Records
# --------------------------------------------
before = df.shape[0]
df.drop_duplicates(inplace=True)
after = df.shape[0]
print(f"\n🧹 Duplicates removed: {before - after}")

# --------------------------------------------
# 3️⃣ Correct Data Types
# --------------------------------------------
# Convert date fields to datetime
for col in ['order_date', 'event_time', 'timestamp']:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
        print(f"📅 Converted {col} to datetime.")

# Convert numerical fields
for col in ['price', 'quantity']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# --------------------------------------------
# 4️⃣ Fix Data Inconsistencies
# --------------------------------------------
# Standardize category column
if 'category' in df.columns:
    df['category'] = df['category'].str.lower().str.strip()

# Remove rows with negative price or quantity
for col in ['price', 'quantity']:
    if col in df.columns:
        df = df[df[col] >= 0]

# --------------------------------------------
# 5️⃣ Handle Outliers (Z-score method or quantile method)
for col in ['price', 'quantity']:
    if col in df.columns:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        print(f"⚠️ {col} outliers detected: {outliers.shape[0]}")
        # Remove outliers (optional)
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

# --------------------------------------------
# 6️⃣ Create New Columns
# --------------------------------------------
# Create Total Revenue
if 'price' in df.columns and 'quantity' in df.columns:
    df['total_revenue'] = df['price'] * df['quantity']

# Extract date features
date_col = None
for col in ['order_date', 'event_time', 'timestamp']:
    if col in df.columns:
        date_col = col
        break

if date_col:
    df['year'] = df[date_col].dt.year
    df['month'] = df[date_col].dt.month
    df['day_of_week'] = df[date_col].dt.day_name()

# --------------------------------------------
# 7️⃣ Save Cleaned Dataset
# --------------------------------------------
df.to_csv("ecommerce_data_final_cleaned.csv", index=False)
print("\n✅ Cleaned dataset saved as 'ecommerce_data_final_cleaned.csv'")
print("📊 Final shape:", df.shape)


🔹 Initial shape: (24999, 8)

🔍 Checking missing values:
date                          0
customer_id                   0
product_category              0
payment_method                0
value [USD]                   0
time_on_site [Minutes]        0
clicks_in_site                0
Unnamed: 7                24999
dtype: int64

🧹 Duplicates removed: 0

✅ Cleaned dataset saved as 'ecommerce_data_final_cleaned.csv'
📊 Final shape: (24999, 8)
